In [ ]:
import os
import sys
import warnings
import dill
import pickle
import pandas
import pyranges
import numpy as np
import pandas as pd

In [ ]:
warnings.filterwarnings("ignore")
# Set stderr to null to avoid strange messages from ray
_stderr = sys.stderr
null = open(os.devnull, 'wb')
work_dir = '/cellar/users/aklie/projects/beta_cell_networks/notebooks/tutorial/infer_grns/scenicplus/pbmc_tutorial'
tmp_dir = '/cellar/users/aklie/tmp/'

In [ ]:
cistopic_obj = dill.load(open(os.path.join(work_dir, 'scATAC/cistopic_obj.pkl'), 'rb'))
region_bin_topics_otsu = pickle.load(open(os.path.join(work_dir, 'scATAC/candidate_enhancers/region_bin_topics_otsu.pkl'), 'rb'))

# CisTopic object

In [ ]:
cistopic_obj.binary_matrix.shape

In [ ]:
cistopic_obj.cell_data

In [ ]:
region_data = cistopic_obj.region_data

In [ ]:
region_data

# Binarized regions

In [ ]:
region_data_filt = region_data[region_data.index.str.contains("chr")]

In [ ]:
all_regions = region_data_filt.index

In [ ]:
topic_regions_pd = pd.Index([])
topic_regions_lst = []
topic_region_mp = {}
for topic, regions in region_bin_topics_otsu.items():
    topic_regions_lst += list(regions.index)
    topic_regions_pd = pd.Index.union(topic_regions_pd, regions.index)
    for region in regions.index:
        topic_region_mp.setdefault(region, []).append(topic)
topic_regions_np = np.array(topic_regions_lst)
topic_regions_set = set(topic_regions_lst)

In [ ]:
len(all_regions)

In [ ]:
len(topic_regions_pd), len(topic_regions_lst), len(topic_regions_np), len(topic_regions_set)

In [ ]:
topic_regions_pd.isin(all_regions).sum()

In [ ]:
non_topic_regions = all_regions[~all_regions.isin(topic_regions_pd)]

In [ ]:
for topic, regions in region_bin_topics_otsu.items():
    print(non_topic_regions.isin(regions.index).sum())

In [ ]:
arr = np.zeros((len(all_regions), len(region_bin_topics_otsu)))
for i, row in enumerate(all_regions):
    if row in topic_region_mp:
        topic_nums = []
        for topic in topic_region_mp[row]:
            #print(i, row, topic, topic.split("Topic")[-1])
            topic_nums.append(int(topic.split("Topic")[-1])-1)
        arr[i, topic_nums] = 1

In [ ]:
arr.sum(axis=0)

In [ ]:
np.array([len(regions) for topic, regions in region_bin_topics_otsu.items()]) == arr.sum(axis=0)

In [ ]:
np.all(all_regions[arr.sum(axis=1) == 0].isin(non_topic_regions))

In [ ]:
np.all(~all_regions[arr.sum(axis=1) == 0].isin(topic_regions_pd))

In [ ]:
arr.sum() == len(topic_regions_lst)

In [ ]:
pr_obj = pyranges.PyRanges(region_data_filt)

In [ ]:
seqs = pyranges.get_fasta(
    pr_obj, 
    "/cellar/users/aklie/projects/beta_cell_networks/notebooks/tutorial/infer_grns/dictys/tutorial/data/genome/genome.fa"
)

In [ ]:
seqs.values[:5], len(seqs)

In [ ]:
len(arr) == len(seqs)

In [ ]:
np.save("_data/topic_labels", arr)

In [ ]:
np.save("_data/seqs", seqs)

In [ ]:
np.save("_data/regions", all_regions.to_numpy())